### Усреднение word2vec

In [1]:
import gensim

unable to import 'smart_open.gcs', disabling that module


In [2]:
from nltk.tokenize import word_tokenize

In [3]:
m = gensim.models.KeyedVectors.load('araneum_none_fasttextskipgram_300_5_2018.model')

In [4]:
m.most_similar('молоко')

[('молока', 0.7558064460754395),
 ('творог', 0.7544727325439453),
 ('пастеризовать', 0.7385150194168091),
 ('кисломолочный', 0.728966474533081),
 ('ряженка', 0.7259336113929749),
 ('кефир', 0.7224420309066772),
 ('коровий', 0.7209818363189697),
 ('простокваша', 0.7202078104019165),
 ('йогурт', 0.713585376739502),
 ('мясо', 0.6996201276779175)]

In [31]:
sw = set(['к', 'на', 'до', 'от', 'и', 'в'])

In [83]:
def get_text_embedding(text):
    tokens = word_tokenize(text)
    token_embeddings = []
    for token in tokens:
        if token in m and token not in sw:
#             print(token)
            token_emb = m[token]
            token_embeddings.append(token_emb)
            
    average_text_emb = sum(token_embeddings) / len(token_embeddings)
#     print('number of embedded words', len(token_embeddings))
#     print('resulting vector size', len(average_text_emb))
    return average_text_emb

In [35]:
text_embedding = get_text_embedding('в москве сегодня солнце')

москве
сегодня
солнце
number of embedded words 3
resulting vector size 300


### С весами TF-IDF

In [36]:
texts = ['в москве сегодня солнце', 'завтра тоже солнце']

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vectorizer = TfidfVectorizer()

In [39]:
vectorizer.fit(texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [71]:
def get_text_embedding_tfidf(text):
    tfidf_vec = vectorizer.transform([text])
    tfidf_vec = tfidf_vec.toarray()[0]
    
    token_embeddings = []
    tokens = word_tokenize(example)
    for token in tokens:
        if token in vectorizer.vocabulary_ and token in m and token not in sw:
            word_index = vectorizer.vocabulary_[token]
            token_score = tfidf_vec[word_index]
            token_emb = m[token] * token_score
            token_embeddings.append(token_emb)

    average_text_emb = sum(token_embeddings) / sum(tfidf_vec)
    return average_text_emb

In [72]:
my_text_emb = get_text_embedding_tfidf('завтра в москве солнце')

In [73]:
my_text_emb.shape

(300,)

### Использовать текст и заголовок

In [78]:
def get_text_embedding_two_texts(text, title, title_weight_ratio=0.7):
    title_emb = get_text_embedding(title)
    text_emb = get_text_embedding(text)
    return title_emb * title_weight_ratio + text_emb * (1 - title_weight_ratio)

In [79]:
text_embedding = get_text_embedding_two_texts('я сегодня ходит в центр москвы и там было солнечно', 'москва и солнце')

москва
солнце
number of embedded words 2
resulting vector size 300
я
сегодня
ходит
центр
москвы
там
было
солнечно
number of embedded words 8
resulting vector size 300


In [80]:
text_embedding.shape

(300,)

### Для классификации

In [7]:
!cp ../sem_01/reviews.csv .

In [8]:
import pandas as pd

In [81]:
df = pd.read_csv('reviews.csv')

In [82]:
df.head()

,label,text
0,1,Эпиграф Добро которое ты делаешь от сердца ты ...
1,1,Теперь это один из моих любимых фильмов в жанр...
2,1,Что скрыто в фильме Лучше не бывает Одна шикар...
3,1,Перед нами очень милое и доброе кино которое л...
4,1,Завязка Мелвин Удал популярный писатель Нет не...


In [86]:
df_batch = df[:2000]

In [87]:
from tqdm import tqdm

In [88]:
text_embeddings = [get_text_embedding(text) for text in tqdm(df_batch.text)]

100%|██████████| 2000/2000 [00:29<00:00, 67.00it/s]


In [89]:
len(text_embeddings)

2000

In [91]:
len(text_embeddings[0])

300

In [92]:
from sklearn.model_selection import train_test_split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(text_embeddings, df_batch.label, random_state=42)

In [95]:
from sklearn.linear_model import LogisticRegression

In [96]:
lr = LogisticRegression()

In [98]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
lr.score(X_test, y_test)

0.818